# クラウドフリー合成

広い範囲の時系列分析では、1ヶ月や1年といった期間の全ての雲と影を取り除いた合成画像を作成することが一般的です。

次の課題は、**クラウドマスキング（雲除去）とメディアン合成（中央値合成）**を組み合わせた、実践的なワークフローです。

# 新しいゴール: 2025年10月のクラウドフリー合成画像を作成する
この課題は以下の2つの新しいステップを要します。

雲除去関数の定義: 画像コレクションの各画像に適用し、雲と影のピクセルを透明にする関数を作成します。

メディアンリダクション: 雲が除去されたコレクション全体を、ピクセルごとの中央値を計算して1枚の画像に合成します。


## ステップ1: 雲除去（Cloud Masking）関数の定義

Landsat 9（LANDSAT/LC09/C02/T1_L2）の画像には、雲や影の情報を格納した専用の品質保証（QA）バンドが含まれています。

雲除去関数を作成するために、まず、そのQAバンドがどれであるかを知る必要があります。

### **Guiding Question: 私たちが使っている LANDSAT/LC09/C02/T1_L2 コレクションのバンド名の中で、雲や影の品質情報を持っているQAバンドの名前は何ですか？**

#### **Guiding Questionの解答**
---
**QA_PIXEL**

---

>GEEでビット単位の操作を行う関数は、そのままズバリ `bitwiseAnd()`です。これは、ビット単位の論理積を計算する関数です。

### bitwiseAnd() の仕組み
**`bitwiseAnd()`** は、2つの数値を2進数（ビット）レベルで比較し、両方の数値で 1 が立っている位置だけを 1 にする演算です。

雲の検出: QA_PIXELの値と、私たちが知りたいビット（例: Bit 3）の値で bitwiseAnd() を取ります。その結果が 0以外 であれば、そのビットが立っている（つまり、雲である）と判断できます。

## 次のステップ: ビットの重みを計算する

>**bitwiseAnd() を使うには、チェックしたいビットの数値的な重みを知る必要があります。ビット番号 $N$ の重みは $2^N$ で計算されます。私たちがチェックしたいのは、Bit 1（Dilated Cloud）と Bit 3（Cloud）です。**

### Guiding Question: これらのビットに対応する数値（重み）はそれぞれいくつになりますか？

#### **Guiding Questionの解答**
---
Bit 1 の重みは 2、Bit 3 の重みは 8 になります。

---

## ステップ2: 雲除去関数の定義

> **これらの重み（2と8）を使用して、以下のPython関数 mask_clouds の中核であるマスク作成ロジックを完成させましょう。**
> **この関数は、画像（image）を受け取り、QA_PIXEL バンドの値が Bit 1 または Bit 3 を含んでいる場合に、そのピクセルを透明にする（マスクする）役割を果たします。**

```python
def mask_clouds(image):
    # 1. QA_PIXELバンドを選択
    qa = image.select('QA_PIXEL')
    
    # 2. ビットの重みを定義
    # Bit 3 (Cloud) の重みは 8、Bit 1 (Dilated Cloud) の重みは 2
    CLOUD_BITS = 8
    DILATED_CLOUD_BITS = 2
    
    # 3. マスクの作成
    # 雲でもなく、かつ、希釈雲でもないピクセル (eq(0)) だけを残すマスクを作成してください。
    mask = # <ここにビット演算と論理積 (And()) を使ったロジックを記述>
           
    # 4. マスクを画像に適用し、雲を除去
    return image.updateMask(mask)
```    

### **Guiding Question**: 上記の mask 変数に、qa.bitwiseAnd(...) と .eq(0).And(...) を使って、「雲（8）でもなく」かつ「希釈雲（2）でもない」ピクセルだけを選択するロジックをどう記述すればよいでしょうか？

### **`bitwiseAnd()` の使用方法**
`qa.bitwiseAnd()` は、選択した QA バンドと、チェックしたいビットの重みを比較します。結果が 0 であれば、そのビットは立っていない（＝クリーンである）と判断できます。

私たちの目標は、「雲（8）でもない」かつ「希釈雲（2）でもない」ピクセルを見つけることです。

以下の構造を使って、mask 変数にロジックを完成させましょう。

#### **Guiding Questionの解答**
---
```python
# 3. マスクの作成
    # 雲 (8) または 希釈雲 (2) が立っているかチェックし、
    # どちらも立っていない (= 0) ピクセルだけを true とします。
    
     mask = qa.bitwiseAnd(8).eq(0).And(  # 雲のチェック: 雲ビット(8)が0(立っていない) AND
             qa.bitwiseAnd(2).eq(0))   # 希釈雲のチェック: 希釈雲ビット(2)が0(立っていない)
```    

---

> このコードでは、以下のことが行われています。
> qa.bitwiseAnd(8): 雲ビットが立っている場所を抽出。
> .eq(0): その結果が **0（雲ではない）**場所を True とします。
> .And(...): 2つの条件（雲ではない かつ 希釈雲ではない）を結合します。
> このロジックを、前のターンで定義した mask_clouds 関数に組み込んで、次のステップに進みましょう。

## ステップ3: 画像コレクション全体に適用

以前フィルタリングした画像コレクションを s2_collection と仮定し、この雲除去関数を適用するには、コレクションのどのメソッドを使用すればよいでしょうか？

#### **関数の完成と適用への準備**

適用するマスクの修正: 画像に適用するのは、計算されたブール値のレイヤー mask です。コレクション全体 (s2_collection) ではありません。

インデント: return 文は、関数 (def) のブロック内（インデントされた位置）にある必要があります。

```python
def mask_clouds(image):
    # 1. QA_PIXELバンドを選択
    qa = image.select('QA_PIXEL')
    
    # 2. 雲の重みを定義
    # Bit 3 (Cloud) = 8, Bit 1 (Dilated Cloud) = 2
    
    # 3. 雲・希釈雲の両方ではないピクセルをTrueとするマスクを作成
    mask = qa.bitwiseAnd(8).eq(0).And(  
             qa.bitwiseAnd(2).eq(0))   
           
    # 4. マスクを画像に適用
    return image.updateMask(mask)
```

## ステップ4: コレクションへの関数の適用

次に、この mask_clouds 関数を、以前フィルタリングした画像コレクション全体に適用する必要があります。GEEで ee.ImageCollection の各要素（画像）に対して同じ操作を行うには、どのメソッドを使用すればよいでしょうか？

>[!warning] **mask_clouds(filtered_collection)** と直接呼び出すのが理想的ですが、GEEでは少し違う方法が必要です。
>私たちが定義した mask_clouds 関数は、単一の画像 (image) を入力として受け取るように設計されています。GEEの ee.ImageCollection は、そのままで>はこの関数を処理できません。

### **コレクションへの関数の適用: `.map()`**

> `.map()` は、リストやコレクションの各要素に関数を適用し、結果を新しいコレクションとして返します。

```python
# 雲除去関数をコレクション内のすべての画像に適用
cloud_free_collection = filtered_collection.map(mask_clouds)
```

>このコードを実行すると、cloud_free_collection という新しいコレクションが生成されます。

### **Guiding Question:** cloud_free_collection には、元の filtered_collection の画像と比べて、どのようなピクセルが含まれていることになりますか？

#### **Guiding Questionの解答**
---
雲除去関数を .map() でコレクション全体に適用したことで、`cloud_free_collection` に含まれる各画像は、Bit 1 (希釈雲) と Bit 3 (雲) が立っていない、すなわちクリーンなピクセルのみが残された状態になります。雲や影のピクセルは透明（マスク）されています。

---

## ステップ5: クラウドフリー合成の作成 (リダクション)

次は、この雲が除去され透明になった複数の画像を組み合わせて、最終的な1枚の合成画像を作成します。透明なピクセルは計算から除外されるため、結果として最もクリーンなピクセルが残ります。

この合成には、コレクション内の全ピクセルについて**中央値（メディアン）**を計算するリダクションメソッドが最もよく使われます。中央値は外れ値（ノイズや一時的な明るさの変化など）の影響を受けにくいためです。

### **Guiding Question:** 雲が除去されたコレクション (cloud_free_collection) に対して、ピクセルごとの中央値を計算し、最終的な合成画像を作成するには、どの Earth Engine メソッドを使用すればよいでしょうか？

***A***: 画像コレクションのすべての画像のピクセルごとの中央値（メディアン）を計算して1枚の画像に合成するには、`.median()` メソッドを使用します。

>[!note] `.median()` は、時間軸方向のリダクション（削減）を実行し、コレクション内の各ピクセルにおいて、マスクされていない（雲ではない）すべての値の中から中央値を計算します。これにより、ノイズや外れ値の影響を受けにくい、非常にクリーンな合成画像が作成されます。

#### **Guiding Questionの解答**
---
```python
# 雲除去されたコレクション全体の中央値を計算し、1枚の合成画像を生成
final_composite = cloud_free_collection.median()
```
---

### **Guiding Question:** 最後に、この final_composite を地図上に表示するための次のステップは何でしょうか？（ヒント: 以前、ベスト画像を地図上に表示するために何を使いましたか？

#### **Guiding Questionの解答**
---
`final_composite.getMapId(vis_params)`

---

## ステップ6: 最終的な合成画像の表示

`getMapId()` メソッドを使うには、画像をどのように描画するか（どのバンドをどの明るさで使うか）を指示する `vis_params` が必要です。

幸い、私たちは前の課題で、`バンド名`、`min`、そして統計的に計算された最適な `max` 値を含む、完璧な `vis_params` を作成済みです。

#### **Guiding Question:** 以前の課題で作成した、最適な vis_params の変数名を覚えていますか？その変数を使って、以下のコードを完成させてください。

```python
vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 
    'min': 0,
    'max': 12408.557332596974, 
    'gamma': 1.3
}
```

> この値には、フィルタリング、雲除去、中央値合成、そして統計分析という、これまで習得した高度なテクニックの全てが詰まっています。この値を適用することで、完全に雲が除去され、コントラストが最適化された合成画像が地図上に表示されます。

## 最終的な地図表示の確認

In [1]:
import ee
import datetime
import folium

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project='earth-change-analysis')

In [4]:
biwako_point = ee.Geometry.Point([136.17, 35.10])
longitude, latitude = biwako_point.coordinates().getInfo() # 地図の中心座標を取得

In [5]:
end_date = '2025-11-17' 
start_date = (datetime.datetime.strptime(end_date, '%Y-%m-%d') - datetime.timedelta(days=365)).strftime('%Y-%m-%d')

In [6]:
l9_collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')

In [7]:
filtered_collection = l9_collection \
    .filterDate(start_date, end_date) \
    .filterBounds(biwako_point)

In [8]:
def mask_clouds(image):
    # 1. QA_PIXELバンドを選択
    qa = image.select('QA_PIXEL')
    
    # 2. 雲の重みを定義
    # Bit 3 (Cloud) = 8, Bit 1 (Dilated Cloud) = 2
    
    # 3. 雲・希釈雲の両方ではないピクセルをTrueとするマスクを作成
    mask = qa.bitwiseAnd(8).eq(0).And(  
             qa.bitwiseAnd(2).eq(0))   
           
    # 4. マスクを画像に適用
    return image.updateMask(mask)


In [9]:
# 雲除去関数をコレクション内のすべての画像に適用
cloud_free_collection = filtered_collection.map(mask_clouds)

# 雲除去されたコレクション全体の中央値を計算し、1枚の合成画像を生成
final_composite = cloud_free_collection.median()

In [10]:
vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 
    'min': 0,
    'max': 12408.557332596974, 
    'gamma': 1.3
}

In [11]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [12]:
my_map = folium.Map(
        location=[latitude, longitude],   # 地図の中心座標
        zoom_start=10,                    # 初期ズームレベル (琵琶湖周辺)
        tiles=basemaps['Google Satellite Hybrid'] # 初期タイルを設定
    )

In [13]:
# (前略: final_composite と my_map が定義されていること)

# 5. GEE 画像レイヤーの追加 (final_composite を使用)
map_id_dict = final_composite.getMapId(vis_params)
tile_url = map_id_dict['tile_fetcher'].url_format

In [14]:
# Foliumにタイルレイヤーを追加
folium.TileLayer(
    tiles=tile_url,
    attr='Google Earth Engine',
    name='Cloud-Free Composite', # レイヤー名を合成画像用に変更
    overlay=True,
    control=True
).add_to(my_map)

In [16]:
# 6. レイヤーコントロールの追加
folium.LayerControl().add_to(my_map)

In [17]:
# 7. マップの表示
display(my_map)